# 噪声模拟器

MindQuantum 中包含各种噪声信道，利用噪声信道我们可以对真实的量子芯片进行模拟。下面介绍如何利用 MindQuantum 完成此任务。

## ChannelAdder

`ChannelAdder` 是一个能够在量子线路特定位置添加特定信道的处理器，例如在测量门后添加比特翻转信道。`ChannelAdder` 类主要由三个函数构成，`_accepter()`、`_excluder()` 和 `_handler(BasicGate)`，其功能对应如下：

- `_accepter()`：返回一个由函数构成的列表，称为接受规则集，其中每个接受规则函数的输入都是一个量子门，当函数返回值为 `True` 时表示我们可以在该量子门后添加信道。

- `_excluder()`：返回一个由函数构成的列表，称为拒绝规则集，其中每个拒绝规则函数的输入都是一个量子门，当函数返回值为 `True` 时表示我们拒绝在该量子门后添加信道。

- `_handler(BasicGate)`：输入一个量子门，返回一段量子线路，表示在输入量子门后添加一段自定义的信道。

我们从定义类 `ChannelAdder` 的 `__call__` 函数，直接调用 `ChannelAdder` 即可生成处理后的量子线路。
下面介绍几中 `ChannelAdder`。

### BitFlipAdder

```python
BitFlipAdder(flip_rate: float = None, with_ctrl=True, device: NaiveChip = None, add_after: bool = True)
```

该 `Adder` 会在所有门后（如果 `add_after` 为 `True`, 否则是在量子门前）添加一个比特翻转信道：

In [1]:
from mindquantum.core.circuit.channel_adder import BitFlipAdder
from mindquantum.core import gates as G
from mindquantum.core.circuit import Circuit

circ = Circuit()+G.H(0)+G.RX('a').on(1)+G.Z(1, 0)
circ.svg()

In [2]:
bit_flip_adder = BitFlipAdder(0.3, with_ctrl=False)
new_circ = bit_flip_adder(circ)
new_circ.svg()

### MeasureAccepter

```python
MeasureAccepter()
```

该 `Adder` 会选择那些测量门，它目前只是一个 `Accepter`，不会改变量子线路中的任何门，需要利用 `MixerAdder`，跟其他的 `Adder` 搭配使用。

### MixerAdder

```python
MixerAdder(adders: typing.List[ChannelAdderBase])
```

`MixerAdder` 可以加多个 `Adder` 混合起来，保证量子门在每一个 `Adder` 中的接受函数集和拒绝函数集同时满足时，顺序添加 `_handler` 产生的量子线路。

举例来说，我们可以将上文提到的 `BitFlipAdder` 和 `MeasureAccepter` 混合起来，达到只在测量门前添加比特翻转信道的功能。

In [3]:
from IPython.display import display_svg
from mindquantum.core.circuit.channel_adder import MixerAdder, MeasureAccepter

mixer = MixerAdder([
    BitFlipAdder(flip_rate=0.01),
    MeasureAccepter(),
], add_after=False)
print(mixer)

circ = Circuit() + G.H(0) + G.RX('a').on(1) + G.Z(1, 0) + G.Measure().on(0)
display_svg(circ.svg())
new_circ = mixer(circ)
new_circ.svg()

MixerAdder<
  BitFlipAdder<flip_rate=0.01, with_ctrl=True>
  MeasureAccepter<>
>


### SequentialAdder

```python
SequentialAdder(adders: typing.List[ChannelAdderBase])
```
`SequentialAdder` 是由多个 `Adder` 顺序构成类，量子线路会经过 `SequentialAdder` 中的 `Adder` 依次处理，生成最终的量子线路。例如，我们想构建一个先在测量门前添加一个 $p=0.01$ 的比特翻转信道，然后在 `q1` 比特上的非测量门和非噪声信道后添加 $p=0.05$ 的比特翻转信道。

我们首先自定义在某个特比特添加比特翻转信道的 `Adder`

In [10]:
from mindquantum.core.circuit.channel_adder import ChannelAdderBase, SequentialAdder, MeasureExcluder, NoiseExcluder
class CustomBitFlipAdder(ChannelAdderBase):
    def __init__(self, q, flip_rate):
        self.q = q
        self.flip_rate = flip_rate
        super().__init__()

    def _accepter(self):
        return [lambda x: self.q in x.obj_qubits or self.q in x.ctrl_qubits]

    def _excluder(self):
        return  [lambda x: isinstance(x, (G.Measure, G.NoiseGate))]

    def _handler(self, g):
        return Circuit([G.BitFlipChannel(self.flip_rate).on(self.q)])

    def __repr__(self):
        return f"CustomBitFlipAdder<q={self.q}, flip_rate={self.flip_rate}>"

seq_adder = SequentialAdder([
    MixerAdder([
        MeasureAccepter(),
        BitFlipAdder(flip_rate=0.01),
    ], add_after=False),
    CustomBitFlipAdder(q=1, flip_rate=0.05),
])
print(seq_adder)

SequentialAdder<
  MixerAdder<
    MeasureAccepter<>
    BitFlipAdder<flip_rate=0.01, with_ctrl=True>
  >
  CustomBitFlipAdder<q=1, flip_rate=0.05>
>


In [11]:
circ = Circuit() + G.H(0) + G.RX('a').on(1) + G.Z(1, 0) + G.Measure().on(0)
display_svg(circ.svg())
new_circ = seq_adder(circ)
new_circ.svg()

## 基于 `ChannelAdder` 的噪声模拟器

我们可以将如上定义的各种 `Adder` 与现有的模拟器组合，构成一个含噪声模拟器。更进一步，我们可以根据定义的芯片参数与拓扑图，生成一个针对该芯片的噪声模拟器。第二种正在开发中，这里先介绍第一种。

In [12]:
from mindquantum.simulator import Simulator
from mindquantum.simulator.noise import NoiseBackend

noiseless_sim = Simulator('mqvector', 2)
noiseless_circ = Circuit().h(0).rx(1.0, 1).z(1, 0).measure(1)
display_svg(noiseless_circ.svg())
res1 = noiseless_sim.sampling(noiseless_circ, shots=10000)
display(res1.svg())



noise_sim = Simulator(NoiseBackend('mqvector', 2, seq_adder))
res2 = noise_sim.sampling(noiseless_circ, shots=10000)
display(res2.svg())
display(noise_sim.backend.transform_circ(noiseless_circ).svg())